Imports

In [1]:
import pandas as pd
import math
import numpy as np
import seaborn
from geopy.distance import vincenty

Get data and basic filters

In [49]:
df_city_raw = pd.read_csv('data/city_large.csv')
df_city_raw = df_city_raw.dropna()

df_county_raw = pd.read_csv('data/location.csv')
del df_county_raw['index']
#df_city.dtypes
# df_city = df_city.drop_duplicates()

Advanced Filtering

In [50]:
def to_float(x):
    x = x.replace(',', '')
    return float(x)
df_city_raw[['Population']] = df_city_raw['Population'].apply(to_float)

Slice of data, cities bigger than ...

In [51]:
cities_bigger_than = [100000, 300000, 500000, 1000000]


Calculate Distances Algorithm

In [ ]:
df_county = df_county_raw

In [52]:
def get_distance(county, df_city, rng):

        p1 = [county['latitude'], county['longitude']]

        # calculate region to search for big cities
        p1x = [county['latitude'] + rng, county['latitude'] - rng]
        p1y = [county['longitude'] + rng, county['longitude'] - rng]

        j = 0
        s_dist = 100000
        for k, city in df_city.iterrows():
            p2 = [city['latitude'], city['longitude']]

            if (p1x[1] < p2[0] < p1x[0]) & (p1y[1] < p2[1] < p1y[0]):
                j += 1

                dist = vincenty(p1, p2).km

                if dist < s_dist:
                    s_dist = dist

        return s_dist, j

In [55]:
df_county = df_county_raw
for bigger_than in cities_bigger_than:
    print 'bigger', bigger_than
    
    df_city = df_city_raw[df_city_raw['Population'] > bigger_than]
    df_county['distance_{}'.format(bigger_than)] = np.nan #create empty column
    
    for i, county in df_county.iterrows():

        for search_region in [5, 10, 15, 20, 30, 50, 100]:
            smaller_distance, j = get_distance(county, df_city, search_region)
            if j:
                break

        df_county['distance_{}'.format(bigger_than)].iloc[i] = smaller_distance
    df_county

bigger 100000
bigger 300000
bigger 500000
bigger 1000000


In [57]:
del df_county['distance_[100000, 300000, 500000, 1000000]']

In [59]:
df_county.to_csv('data/county_distances.csv', index=False)